In [4]:
from __future__ import division
from __future__ import print_function
import time
import tensorflow as tf
import numpy as np


In [6]:
#set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)
import sys, os
import numpy as np
import time
import csv
from sklearn.metrics import confusion_matrix
import scipy.io as sio
import random
import ABIDE_Parser as Reader



ImportError: No module named 'ABIDE_Parser'